 # Genetic Algorithm Solver for Pokedle

In [1]:
import pandas as pd
import random
import numpy as np

# Load dataset
df = pd.read_csv("03_cleaned_with_images_and_evolutionary_stages.csv")

ATTRIBUTES = ['Generation', 'Height', 'Weight', 'Type1', 'Type2', 'Color', 'evolutionary_stage']


 # Feedback Function (reuse from CSP)

In [2]:
def get_feedback(secret, guess, attributes, numeric_attrs=['Height', 'Weight']):
    """Generate feedback only for the specified attributes."""
    feedback = {}
    
    # Get secret and guess types (handling NaN)
    secret_types = {secret.get('Type1'), secret.get('Type2')} - {None, float('nan')}
    secret_types = {t for t in secret_types if not (isinstance(t, float) and pd.isna(t))}
    
    guess_types = {guess.get('Type1'), guess.get('Type2')} - {None, float('nan')}
    guess_types = {t for t in guess_types if not (isinstance(t, float) and pd.isna(t))}
    
    for attr in attributes:
        if attr == 'image_url':
            continue
        
        # Handle Type1 and Type2 specially
        if attr in ['Type1', 'Type2']:
            guess_value = guess[attr]
            secret_value = secret[attr]
            
            if pd.isna(guess_value) and pd.isna(secret_value):
                feedback[attr] = 'green'
            elif pd.isna(guess_value) or pd.isna(secret_value):
                feedback[attr] = 'gray'
            elif guess_value == secret_value:
                feedback[attr] = 'green'
            elif guess_value in secret_types:
                feedback[attr] = 'yellow'
            else:
                feedback[attr] = 'gray'
        elif pd.isna(secret[attr]) or pd.isna(guess[attr]):
            feedback[attr] = 'gray'
        elif secret[attr] == guess[attr]:
            feedback[attr] = 'green'
        elif attr in numeric_attrs:
            if guess[attr] < secret[attr]:
                feedback[attr] = 'higher'
            else:
                feedback[attr] = 'lower'
        else:
            feedback[attr] = 'gray'
    
    return feedback


 # GA Core Classes

In [3]:
class PokedleGA:
    def __init__(self, dataframe, attributes=ATTRIBUTES, 
                 pop_size=50, elite_size=10, mutation_rate=0.02):
        self.df = dataframe.copy()
        self.attributes = attributes
        self.pop_size = pop_size
        self.elite_size = elite_size
        self.mutation_rate = mutation_rate
        
        # Initialize population with random Pokemon indices
        self.population = self.df.sample(pop_size).index.tolist()
        self.feedback_history = []  # Store all previous feedback
        
    def fitness(self, pokemon_idx):
        """
        Calculate fitness based on how well this Pokemon matches all feedback received.
        Higher score = better match with feedback constraints.
        """
        pokemon = self.df.loc[pokemon_idx]
        score = 0
        
        for guess_idx, feedback in self.feedback_history:
            guess = self.df.loc[guess_idx]
            
            for attr, status in feedback.items():
                if attr == 'image_url':
                    continue
                
                # Green: exact match required
                if status == 'green':
                    if pokemon[attr] == guess[attr]:
                        score += 10  # High reward for matching green
                    else:
                        score -= 20  # Heavy penalty for violating green
                
                # Gray: must NOT match
                elif status == 'gray':
                    if attr in ['Type1', 'Type2']:
                        # For types, gray means this type shouldn't appear anywhere
                        if guess[attr] not in [pokemon['Type1'], pokemon['Type2']]:
                            score += 5
                        else:
                            score -= 10
                    else:
                        if pokemon[attr] != guess[attr]:
                            score += 5
                        else:
                            score -= 10
                
                # Yellow: type exists but wrong position
                elif status == 'yellow':
                    pokemon_types = {pokemon['Type1'], pokemon['Type2']} - {None, float('nan')}
                    if guess[attr] in pokemon_types and pokemon[attr] != guess[attr]:
                        score += 8
                    else:
                        score -= 10
                
                # Higher/Lower: numeric constraints
                elif status == 'higher':
                    if pokemon[attr] > guess[attr]:
                        # Bonus for being closer to the right range
                        score += 10
                    else:
                        score -= 15
                
                elif status == 'lower':
                    if pokemon[attr] < guess[attr]:
                        score += 10
                    else:
                        score -= 15
        
        return max(0, score)  # Don't return negative fitness
    
    def select_parents(self):
        """Tournament selection: pick best from random subset."""
        tournament_size = 5
        tournament = random.sample(self.population, tournament_size)
        tournament_fitness = [(idx, self.fitness(idx)) for idx in tournament]
        tournament_fitness.sort(key=lambda x: x[1], reverse=True)
        return tournament_fitness[0][0], tournament_fitness[1][0]
    
    def crossover(self, parent1_idx, parent2_idx):
        """
        Create child by mixing attributes from two parents.
        Child is a new Pokemon whose attributes come from parents.
        """
        parent1 = self.df.loc[parent1_idx]
        parent2 = self.df.loc[parent2_idx]
        
        # Find a Pokemon in dataset that matches the mixed attributes
        # For simplicity, randomly pick attributes from each parent
        # and find closest match in dataset
        
        target_attrs = {}
        for attr in self.attributes:
            target_attrs[attr] = parent1[attr] if random.random() < 0.5 else parent2[attr]
        
        # Find Pokemon that matches most of these attributes
        best_match_idx = None
        best_match_score = -1
        
        # Sample subset for efficiency
        candidates = self.df.sample(min(100, len(self.df)))
        
        for idx, row in candidates.iterrows():
            match_score = sum(1 for attr in self.attributes if row[attr] == target_attrs[attr])
            if match_score > best_match_score:
                best_match_score = match_score
                best_match_idx = idx
        
        return best_match_idx
    
    def mutate(self, pokemon_idx):
        """Replace with random Pokemon from dataset."""
        if random.random() < self.mutation_rate:
            return self.df.sample(1).index[0]
        return pokemon_idx
    
    def evolve(self):
        """Run one generation of evolution."""
        # Calculate fitness for all
        fitness_scores = [(idx, self.fitness(idx)) for idx in self.population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Keep elite
        new_population = [idx for idx, _ in fitness_scores[:self.elite_size]]
        
        # Generate rest through crossover
        while len(new_population) < self.pop_size:
            parent1, parent2 = self.select_parents()
            child = self.crossover(parent1, parent2)
            child = self.mutate(child)
            new_population.append(child)
        
        self.population = new_population
    
    def best_guess(self):
        """Return the Pokemon with highest fitness."""
        fitness_scores = [(idx, self.fitness(idx)) for idx in self.population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        return self.df.loc[fitness_scores[0][0]]
    
    def update_feedback(self, guess_idx, feedback):
        """Store feedback from latest guess."""
        self.feedback_history.append((guess_idx, feedback))


 # GA Solver Runner

In [4]:
def solve_ga(df, attributes=ATTRIBUTES, max_attempts=10, generations_per_guess=20):
    """
    Solve Pokedle using Genetic Algorithm.
    
    Args:
        generations_per_guess: How many GA generations to evolve before making each guess
    """
    secret = df.sample(1).iloc[0]
    solver = PokedleGA(df, attributes)
    
    print(f"(DEBUG) Secret Pokémon: {secret['Original_Name']}\n")
    
    for attempt in range(1, max_attempts + 1):
        # Evolve population for several generations
        for gen in range(generations_per_guess):
            solver.evolve()
        
        # Make best guess
        guess = solver.best_guess()
        guess_idx = guess.name
        
        feedback = get_feedback(secret, guess, attributes=attributes, numeric_attrs=['Height', 'Weight'])
        
        print(f"Attempt {attempt}: {guess['Original_Name']}")
        print("Feedback:", feedback)
        
        # Check win condition
        non_image_feedback = {k: v for k, v in feedback.items() if k != 'image_url'}
        if all(v == 'green' for v in non_image_feedback.values()):
            print(f"🎯 Guessed correctly! The Pokémon was {guess['Original_Name']}.")
            return attempt
        
        # Update GA with new feedback
        solver.update_feedback(guess_idx, feedback)
        
        # Show population diversity
        unique_pokemon = len(set(solver.population))
        print(f"Population diversity: {unique_pokemon}/{solver.pop_size} unique Pokemon\n")
    
    print(f"❌ Max attempts reached. Secret was: {secret['Original_Name']}")
    return max_attempts + 1


 # Benchmark GA

In [ ]:
import time

def benchmark_ga(num_runs=100, max_attempts=10, generations_per_guess=20):
    """Benchmark the GA solver."""
    attempts_list = []
    
    print(f"🚀 Running GA benchmark with {num_runs} runs...")
    print(f"   Pop size: 50, Generations per guess: {generations_per_guess}\n")
    
    start_time = time.time()
    
    for i in range(num_runs):
        if (i + 1) % 10 == 0:
            print(f"Progress: {i + 1}/{num_runs}")
        result = solve_ga(df, ATTRIBUTES, max_attempts, generations_per_guess)
        attempts_list.append(result)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Calculate statistics
    avg_attempts = sum(attempts_list) / len(attempts_list)
    min_attempts = min(attempts_list)
    max_attempts_taken = max(attempts_list)
    success_rate = sum(1 for a in attempts_list if a <= max_attempts) / len(attempts_list) * 100
    
    print(f"\n{'='*60}")
    print("📊 GA BENCHMARK RESULTS")
    print(f"{'='*60}")
    print(f"Average attempts: {avg_attempts:.2f}")
    print(f"Min attempts: {min_attempts}")
    print(f"Max attempts: {max_attempts_taken}")
    print(f"Success rate: {success_rate:.1f}%")
    print(f"Total time: {elapsed_time:.2f}s")
    print(f"Avg time/solve: {elapsed_time/num_runs:.4f}s")
    print(f"{'='*60}\n")
    
    return {
        'avg_attempts': avg_attempts,
        'success_rate': success_rate,
        'time_per_solve': elapsed_time / num_runs
    }


: 

 # Run Benchmark

In [ ]:
# Quick test
# ga_results = benchmark_ga(num_runs=10, max_attempts=10, generations_per_guess=20)

# Full benchmark
ga_results = benchmark_ga(num_runs=100, max_attempts=10, generations_per_guess=20)


🚀 Running GA benchmark with 100 runs...
   Pop size: 50, Generations per guess: 20

(DEBUG) Secret Pokémon: Crocalor

Attempt 1: Naclstack
Feedback: {'Generation': 'green', 'Height': 'higher', 'Weight': 'lower', 'Type1': 'gray', 'Type2': 'green', 'Color': 'gray', 'evolutionary_stage': 'green'}
Population diversity: 48/50 unique Pokemon

Attempt 2: Scovillain
Feedback: {'Generation': 'green', 'Height': 'higher', 'Weight': 'higher', 'Type1': 'gray', 'Type2': 'gray', 'Color': 'gray', 'evolutionary_stage': 'green'}
Population diversity: 34/50 unique Pokemon

Attempt 3: Kilowattrel
Feedback: {'Generation': 'green', 'Height': 'lower', 'Weight': 'lower', 'Type1': 'gray', 'Type2': 'gray', 'Color': 'gray', 'evolutionary_stage': 'green'}
Population diversity: 33/50 unique Pokemon

Attempt 4: Quaxwell
Feedback: {'Generation': 'green', 'Height': 'lower', 'Weight': 'higher', 'Type1': 'gray', 'Type2': 'green', 'Color': 'gray', 'evolutionary_stage': 'green'}
Population diversity: 32/50 unique Pokemon

 # Compare with CSP



 You can now compare:

 - CSP: ~4.1 attempts average (from your previous results)

 - GA: See results above



 GA Characteristics:

 - More exploration-based

 - Can escape local optima

 - Slower per iteration but more flexible

 - Works well when constraints are complex or noisy